In [1]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path = 'C:/Users/Sahil/.wdm/drivers/chromedriver/win32/110.0.5481/chromedriver.exe')
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="ee8ba579cbcf8051cb78aa824d30fe1c")>

In [9]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

In [7]:
class IEDB_MHC1_Immunogenicity:
    def __init__(self):
        self.base_url = 'http://tools.iedb.org/immunogenicity/'
      
    def get_data(self, source_filepath):
        epit_list = []
        length_list = []
        immuno_score_list = []
    
    #home page
        driver.get(self.base_url)
        sleep(3)
    
    #choose file
        choose_file = driver.find_element_by_xpath('//*[@id="id_sequence_file"]')
        choose_file.send_keys(source_filepath)
    
    #submit button
        submit = driver.find_element_by_xpath('//*[@id="input-form"]/table/tbody/tr[6]/th/div/input[1]')
        submit.click()
    
    #wait until output table appears
        WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="result_table"]')))

    #finding all elements from the table
        epit = driver.find_elements_by_class_name('sequence')
        length = driver.find_elements_by_xpath('//*[@id="result_table"]/tbody/tr/td[2]')  
        score = driver.find_elements_by_xpath('//*[@id="result_table"]/tbody/tr/td[3]')
    
    #adding them into list
        for ep, leng, sco in zip(epit, length, score):
            epit_list.append(ep.text)
            length_list.append(int(leng.text))
            immuno_score_list.append(float(sco.text))
        
   #close the window     
        driver.close()
    
    #return dataframe
        df = pd.DataFrame({'Epitope':epit_list, 'Length':length_list, 'Immunogenicity':immuno_score_list})
        return df
    
    def save_results(self, df, dest_filepath):
        df.to_csv(dest_filepath, index = False)
        

In [5]:
source_filepath = r'C:\Users\Sahil\Omdena\Development of Vaccines for JC Virus\9per_peptides.csv'
dest_filepath = r'C:\Users\Sahil\Omdena\Development of Vaccines for JC Virus\IEDB_MHC1_immunogenicity_results.csv'

In [10]:
im = IEDB_MHC1_Immunogenicity()
data = im.get_data(source_filepath)
data

,Epitope,Length,Immunogenicity
0,LLLIWFRPV,9,0.49342
1,EPLVWIDCY,9,0.36614
2,SPLVWIDCY,9,0.36614
3,ILMWEAVTL,9,0.35774
4,LLMWEAVTV,9,0.35774
...,...,...,...
60,AYLRKCKEF,9,-0.33577
61,NLSLMRKAY,9,-0.34628
62,APKKPKEPV,9,-0.42278
63,CLLPKMDSV,9,-0.46800


In [11]:
im.save_results(data, dest_filepath)